In [0]:
print('Unmounting all mounts beginning with /mnt/')
dbutils.fs.mounts()
for mount in dbutils.fs.mounts():
    if mount.mountPoint.startswith('/mnt/'):
        dbutils.fs.unmount(mount.mountPoint)

# Re-list all mount points
print('Re-listing all mounts')
dbutils.fs.mounts()

Unmounting all mounts beginning with /mnt/
/mnt/churnfiles has been unmounted.
Re-listing all mounts
Out[1]: [MountInfo(mountPoint='/databricks-datasets', source='databricks-datasets', encryptionType=''),
 MountInfo(mountPoint='/databricks/mlflow-tracking', source='databricks/mlflow-tracking', encryptionType=''),
 MountInfo(mountPoint='/databricks-results', source='databricks-results', encryptionType=''),
 MountInfo(mountPoint='/databricks/mlflow-registry', source='databricks/mlflow-registry', encryptionType=''),
 MountInfo(mountPoint='/', source='DatabricksRoot', encryptionType='')]

In [0]:
dbutils.fs.mount(
  source = "wasbs://target@mygkstorage.blob.core.windows.net",
  mount_point = "/mnt/churnfiles",
  extra_configs = {"fs.azure.account.key.mygkstorage.blob.core.windows.net":"0PrpnjruhtHMc5e/FtWuMPlrnxP2qRSKDjv9hek/uUXoXDlf4bO0UukBHjKD/C5viZDvmKVd1Bn++ASthrkGVg=="}
                    )


Out[1]: True

In [0]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

### KPI

**1) Top 5 product types with highest positive and negative Feedback**

In [0]:
srs = spark.read.csv("/mnt/churnfiles/srcCleaned.csv", header="True",inferSchema=True)

In [0]:
product = spark.read.csv("/mnt/churnfiles/productClean.csv", header="True",inferSchema=True)

In [0]:
product.createOrReplaceTempView("productTable")
srs.createOrReplaceTempView("srsTable")



In [0]:
%sql
select p.product_type `Proudct Type`, count(sr.feedback_type) `Total Positive Feedback`
from productTable p inner join srsTable sr 
on sr.product_id = p.product_id
group by p.product_type,sr.feedback_type
having sr.feedback_type ='positive'
order by `Total Positive Feedback` desc
limit 5

Proudct Type,Total Positive Feedback
CREDIT CARD,275
SAVINGS ACCOUNT,206
LIFE INSURANCE,177
MOTOR INSURANCE,177
HEALTH INSURANCE,174


In [0]:
%sql
select p.product_type `Proudct Type`, count(sr.feedback_type) `Total Negative Feedback`
from productTable p inner join srsTable sr 
on sr.product_id = p.product_id
group by p.product_type,sr.feedback_type
having sr.feedback_type ='negative'
order by `Total Negative Feedback` desc
limit 5

Proudct Type,Total Negative Feedback
CREDIT CARD,111
SAVINGS ACCOUNT,90
HEALTH INSURANCE,67
FIXED DEPOSIT,61
HOME LOAN,57


**2) Identitfy channels through which highest positive feedback were received - Order of highget to lowest**

In [0]:
%sql
select channel `Channel`, count(feedback_type) `Total Positive Feedback`
from srsTable
group by channel,feedback_type
having feedback_type ='positive'

Channel,Total Positive Feedback
Net Banking,667
Phone Banking,665
Branch,689


In [0]:
%sql
select channel `Channel`, count(feedback_type) `Total Negative Feedback`
from srsTable
group by channel,feedback_type
having feedback_type ='negative'

Channel,Total Negative Feedback
Branch,238
Phone Banking,251
Net Banking,255


**3) Show top 5 product type that is preformaing well where type_deposit_withdrawal='D' along with top channels used(order highgest to lowest)**

In [0]:
transact = spark.read.csv("/mnt/churnfiles/transactionclean.csv", header="True",inferSchema=True)

In [0]:
transact.createOrReplaceTempView("transactionTable")

In [0]:
%sql
select product_type `Product Type`, first(channel) `Channel`, sum(deposit) `Deposit` from
(select p.product_type, sum(t.transaction_amount) `deposit`,t.channel,t.type_deposit_withdrawal
from productTable p inner join transactionTable t
on p.product_id = t.product_id
group by p.product_type, t.channel, t.type_deposit_withdrawal
having t.type_deposit_withdrawal = 'D'
order by `deposit` desc) sub
group by product_type order by `Deposit` desc
limit 5

Product Type,Channel,Deposit
HEALTH INSURANCE,Branch,11454267
MOTOR INSURANCE,Net Banking,11115270
HOME LOAN,Net Banking,10924311
LIFE INSURANCE,Net Banking,10470167
HOME INSURANCE,Branch,10239066


**4) Show top 5 product type that is preformaing low where type_deposit_withdrawal='W' along with top channels used(order highgest to lowest)**

In [0]:
%sql
select product_type `Product Type`, first(channel) `Channel`, sum(withdraw) `Withdrawal` from
(select p.product_type, sum(t.transaction_amount) `withdraw`,t.channel,t.type_deposit_withdrawal
from productTable p inner join transactionTable t
on p.product_id = t.product_id
group by p.product_type, t.channel, t.type_deposit_withdrawal
having t.type_deposit_withdrawal = 'W'
order by `withdraw` desc) sub
group by product_type order by `Withdrawal` desc
limit 5

Product Type,Channel,Withdrawal
CREDIT CARD,Branch,9953294
SAVINGS ACCOUNT,Net Banking,6360715
EQUITIES,Branch,4236251
CURRENT ACCOUNT,Net Banking,2150628
MUTUAL FUNDS,Branch,1709005


**5) Show top 5 product type that is preferred by low, medium, high income group customer**

In [0]:
customer = spark.read.csv("/mnt/churnfiles/customerClean.csv", header="True",inferSchema=True)

In [0]:
customer.createOrReplaceTempView("customerTable")

In [0]:
%sql
select p.product_type,c.income_level,count(c.income_level) `Income level count`
from productTable p inner join srsTable sr
on p.product_id = sr.product_id
inner join customerTable c
on sr.customer_id = c.customer_id
where sr.feedback_type = 'positive'
group by p.product_type, c.income_level
having income_level='High'
order by c.income_level, `Income level count` desc
limit 5


product_type,income_level,Income level count
CREDIT CARD,High,89
LIFE INSURANCE,High,66
SAVINGS ACCOUNT,High,58
MOTOR INSURANCE,High,56
FIXED DEPOSIT,High,51


In [0]:
%sql
select p.product_type,c.income_level,count(c.income_level) `Income level count`
from productTable p inner join srsTable sr
on p.product_id = sr.product_id
inner join customerTable c
on sr.customer_id = c.customer_id
where sr.feedback_type = 'positive'
group by p.product_type, c.income_level
having income_level='Medium'
order by c.income_level, `Income level count` desc
limit 5

product_type,income_level,Income level count
CREDIT CARD,Medium,94
SAVINGS ACCOUNT,Medium,70
HEALTH INSURANCE,Medium,67
MOTOR INSURANCE,Medium,62
HOME LOAN,Medium,57


In [0]:
%sql
select p.product_type,c.income_level,count(c.income_level) `Income level count`
from productTable p inner join srsTable sr
on p.product_id = sr.product_id
inner join customerTable c
on sr.customer_id = c.customer_id
where sr.feedback_type = 'positive'
group by p.product_type, c.income_level
having income_level='Low'
order by c.income_level, `Income level count` desc
limit 5

product_type,income_level,Income level count
CREDIT CARD,Low,92
SAVINGS ACCOUNT,Low,78
LIFE INSURANCE,Low,64
HEALTH INSURANCE,Low,64
HOME INSURANCE,Low,62


**6) Which states warrent the most positive, negative complaints and what product are being sold in those states?**

In [0]:
address = spark.read.csv("/mnt/churnfiles/addressClean.csv", header="True",inferSchema=True)

In [0]:
address.createOrReplaceTempView("addressTable")

In [0]:
%sql

select state, sum(count_feedback) `No of Positive Feedback`,first(product_name) `Product Name` from
(select a.state state, sr.feedback_type, count(sr.feedback_type) count_feedback, p.product_name product_name
from addressTable a inner join srsTable sr
on sr.customer_id = a.customer_id
inner join productTable p
on p.product_id = sr.product_id
group by a.state,sr.feedback_type,p.product_name
having sr.feedback_type = 'positive') sub
group by state
order by `No of Positive Feedback` desc
limit 5



state,No of Positive Feedback,Product Name
California,231,Equities
Texas,204,Health Security Plan
Florida,146,Foreign Education Loan
Virginia,108,Regular Fixed Deposit
New York,99,Central Govt Scheme


In [0]:
%sql

select state, sum(count_feedback) `No of Negative Feedback`,first(product_name) `Product Name` from
(select a.state state, sr.feedback_type, count(sr.feedback_type) count_feedback, p.product_name product_name
from addressTable a inner join srsTable sr
on sr.customer_id = a.customer_id
inner join productTable p
on p.product_id = sr.product_id
group by a.state,sr.feedback_type,p.product_name
having sr.feedback_type = 'negative') sub
group by state
order by `No of Negative Feedback` desc
limit 5

state,No of Negative Feedback,Product Name
New York,62,Super Saver Facility
Florida,58,Life Protect Plus Plan
Texas,57,Jet Privilege World
California,55,Ergo Home Insurance
District of Columbia,39,Max Current Account


**7) How many accounts are currently open and closed?(Ratio of open and close)**

In [0]:
%sql

select account_status, count(account_status) `No of account status`
from productTable
group by account_status


account_status,No of account status
closed,417
open,4478


In [0]:
%sql

select (select count(*) open from productTable WHERE account_status = 'open')/(select count(*) closed from productTable WHERE account_status = 'closed') `Ratio of open and closed`
from productTable limit 1

Ratio of open and closed
10.738609112709833
